In [1]:
from IPython.display import display, HTML
from bokeh.io import show
from bokeh.models import LogColorMapper, ColumnDataSource
from bokeh.palettes import Cividis256 as palette
from bokeh.plotting import figure
from bokeh.sampledata.us_counties import data as counties
import pandas as pd

#Import data file
df = pd.read_csv('/Users/Valerie/cali_data.csv',delimiter=',', header=0, skiprows=2)

#Drop first row
df = df.drop(df.index[0])

#Rename specific columns
df2 = df.rename(columns={'CA County':'county_name', 'Poverty Percent, All Ages': 'poverty_percent', 'Median Household Income': 'median_income' },inplace=True)

#Select columns
df_factors = df[["county_name", "poverty_percent", "median_income"]]

#Convert to dictionary
df_dic = df_factors.to_dict('index')


display(HTML(df_factors.to_html()))


,county_name,poverty_percent,median_income
1,Alameda County,8.9,"107,589"
2,Alpine County,17.2,"58,112"
3,Amador County,9.8,"62,640"
4,Butte County,16.1,"58,394"
5,Calaveras County,12.1,"68,248"
6,Colusa County,12.0,"59,048"
7,Contra Costa County,7.9,"106,555"
8,Del Norte County,17.9,"48,979"
9,El Dorado County,8.7,"86,202"
10,Fresno County,20.5,"56,926"


In [2]:
palette = tuple(reversed(palette))

counties = {
    code: county for code, county in counties.items() if county["state"] == "ca"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]


#Merge the two dicts
def merge_two_dicts(a_dict, b_dict):
    # iterate thru keys of each dict in order (using python 3) 
    # assuming dicts are same length
    for idx in range(len(b_dict)):

        a_key = list(a_dict.keys())[idx]
        b_key = list(b_dict.keys())[idx]

        if('median_income' in a_dict[a_key]):
            b_dict[b_key]['median_income'] = a_dict[a_key]['median_income']

        if('poverty_percent' in a_dict[a_key]):
            b_dict[b_key]['poverty_percent'] = a_dict[a_key]['poverty_percent']
           
    return b_dict

a_dict = df_dic
b_dict = counties

#Overwrite previous value of counties
counties= merge_two_dicts(a_dict, b_dict)

print(counties)


{(6, 1): {'name': 'Alameda', 'detailed name': 'Alameda County, California', 'state': 'ca', 'lats': [37.90194, 37.89909, 37.8965, 37.8941, 37.88703, 37.87532, 37.85151, 37.84128, 37.83271, 37.81342, 37.80299, 37.77014, 37.71889, 37.72341, 37.72731, 37.73772, 37.7591, 37.78963, 37.81216, 37.81732, 37.81722, 37.81707, 37.81576, 37.79884, 37.77296, 37.76619, 37.74655, 37.74333, 37.73377, 37.71473, 37.70453, 37.68538, 37.67044, 37.63563, 37.61667, 37.57664, 37.55004, 37.53253, 37.52553, 37.51941, 37.50493, 37.49883, 37.49563, 37.49245, 37.48969, 37.48568, 37.48239, 37.48178, 37.48217, 37.4822, 37.48243, 37.48263, 37.48283, 37.48407, 37.48419, 37.48462, 37.48007, 37.47492, 37.4714, 37.47082, 37.46741, 37.46479, 37.46373, 37.46167, 37.4591, 37.45484, 37.45712, 37.45684, 37.46027, 37.46435, 37.46796, 37.46264, 37.46709, 37.46134, 37.47631, 37.47784, 37.4818, 37.49806, 37.50487, 37.50671, 37.50685, 37.51235, 37.52282, 37.54353, 37.54746, 37.55745, 37.58133, 37.59108, 37.5929, 37.60031, 37.60337

In [3]:
county_names = [county['name'] for county in counties.values()]
poverty_est = [county['poverty_percent'] for county in counties.values()]
median_income = [county['median_income'] for county in counties.values()]
color_mapper = LogColorMapper(palette=palette)


data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    poverty=poverty_est,
    income = median_income
    
)

print(data)


{'x': [[-122.26254, -122.25876, -122.25586, -122.25309, -122.24897, -122.22138, -122.20723, -122.19419, -122.18501, -122.16687, -122.11766, -122.00433, -121.95961, -121.9403, -121.92436, -121.87996, -121.79189, -121.66954, -121.58156, -121.557, -121.55694, -121.55695, -121.55705, -121.55696, -121.557, -121.55686, -121.55715, -121.55696, -121.55706, -121.55706, -121.55697, -121.55696, -121.55696, -121.55687, -121.55726, -121.55689, -121.55666, -121.54575, -121.51665, -121.50105, -121.49605, -121.48104, -121.47748, -121.47294, -121.46927, -121.47051, -121.47161, -121.47192, -121.47265, -121.47271, -121.53223, -121.62135, -121.66174, -121.76889, -121.81962, -121.86339, -121.87061, -121.87626, -121.88054, -121.88803, -121.89565, -121.90206, -121.90672, -121.91106, -121.9158, -121.92642, -121.92779, -121.93271, -121.93358, -121.94051, -121.94678, -121.95558, -121.99825, -122.04339, -122.07905, -122.08147, -122.0871, -122.10993, -122.11568, -122.11723, -122.11735, -122.12186, -122.13041, -12

In [4]:
TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="CA Poverty and Median Household Income, 2019", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("Poverty Percentage", "@poverty%"), ('Median Household Income', "@income"), ("(Long, Lat)", "($x, $y)")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('x', 'y', source=data,
          fill_color={'field': 'poverty', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

show(p)